<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/vector_mora_v2_endo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Download Dataset
!mkdir /content/PitVQA/datasets
%cd /content/PitVQA/datasets
!gdown --id 1FoAEY_u0PTAlrscjEifi2om15A83wL78

# Unzipping the VQA EndoVis18 Dataset
!unzip -q EndoVis-18-VQA.zip

mkdir: cannot create directory ‘/content/PitVQA/datasets’: No such file or directory
[Errno 2] No such file or directory: '/content/PitVQA/datasets'
/content
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1FoAEY_u0PTAlrscjEifi2om15A83wL78
From (redirected): https://drive.google.com/uc?id=1FoAEY_u0PTAlrscjEifi2om15A83wL78&confirm=t&uuid=e6d90e00-1461-49c2-8e37-8ecbea0786f5
To: /content/EndoVis-18-VQA.zip
100% 2.71G/2.71G [00:54<00:00, 49.9MB/s]


In [ ]:
!pip install -q timm==0.9.12 fairscale==0.4.13 scikit-learn==1.3.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 25.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 81.3 MB/s eta 0:00:00


In [ ]:
# dataloader
import os
import glob

from PIL import Image
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from torchvision.transforms.functional import InterpolationMode
from pathlib import Path

# model
import torch
import torch.nn as nn

from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import ViTModel, BlipConfig, BlipTextModel

# main
import os
import torch
import argparse
import torch.utils.data
import numpy as np
import random

from torch import nn
from torch.utils.data import DataLoader
from transformers import GPT2Tokenizer

from tqdm import tqdm
from nltk.translate.bleu_score import corpus_bleu

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Dataloader

In [ ]:
class EndoVis18VQAGen(Dataset):
    def __init__(self, seq, folder_head, folder_tail):
        self.transform = transforms.Compose([
            transforms.Resize((224, 224), interpolation=InterpolationMode.BICUBIC),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        # files, question and answers
        filenames = []
        for curr_seq in seq:
            filenames = filenames + glob.glob(folder_head + str(curr_seq) + folder_tail)
        self.vqas = []
        for file in filenames:
            file_data = open(file, "r")
            lines = [line.strip("\n") for line in file_data if line != "\n"]
            file_data.close()
            for line in lines:
                self.vqas.append([file, line])
        print('Total files: %d | Total question: %.d' % (len(filenames), len(self.vqas)))

    def __len__(self):
        return len(self.vqas)

    def __getitem__(self, idx):
        qa_full_path = Path(self.vqas[idx][0])
        seq_path = qa_full_path.parents[2]
        file_name = self.vqas[idx][0].split('/')[-1]  # / in linux and \\ in windows

        # img
        img_loc = os.path.join(seq_path, 'left_fr', file_name.split('_')[0] + '.png')
        raw_image = Image.open(img_loc).convert('RGB')
        img = self.transform(raw_image)

        # question and answer
        question, answer = self.vqas[idx][1].split('|')

        return img, question, answer

### Utils

In [ ]:
class AverageMeter(object):
    """
    Keeps track of most recent, average, sum, and count of a metric.
    """

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def adjust_learning_rate(optimizer, shrink_factor):
    print("\nDECAYING learning rate.")
    for param_group in optimizer.param_groups:
        param_group['lr'] = param_group['lr'] * shrink_factor
    print("The new learning rate is %f\n" % (optimizer.param_groups[0]['lr'],))

### Model

In [ ]:
class Vector_MoRA(nn.Module):
    def __init__(self, w_qkv, mora_rank, lora_dropout):
        super().__init__()
        self.base_layer = w_qkv  # original c_atten layer
        self.r = mora_rank  # one of mora rank elements in the list
        self.in_features = w_qkv.weight.shape[0]  # 768
        self.out_features = w_qkv.weight.shape[1]  # 2304

        # LoRA dropout
        self.lora_dropout = nn.ModuleDict({
            'default': nn.Dropout(p=lora_dropout)
        })

        # LoRA A and B matrices
        self.lora_A = nn.ModuleDict({
            'default': nn.Conv1d(self.r, self.r, bias=False, kernel_size=1)
        })
        # self.lora_A = nn.ModuleDict({
        #     'default': nn.Sequential(
        #         nn.Conv1d(self.r, self.r, bias=False, kernel_size=1),
        #         nn.ReLU(),
        #         nn.Linear(self.r, self.r, bias=False)
        #     )
        # })
        nn.init.zeros_(self.lora_A['default'].weight)
        self.lora_B = self.lora_A  # not for use

        # For Embedding layer
        self.lora_embedding_A = nn.ParameterDict({})
        self.lora_embedding_B = nn.ParameterDict({})

    def forward(self, x):  # [32, 32, 768]
        # Original output
        result = self.base_layer(x)  # [32, 32, 2304]
        x = self.lora_dropout['default'](x)  # x is the input for mora

        '''apply compression before lora_A'''  # RoPE
        in_f, out_f = self.in_features, self.out_features
        r = self.r
        # suppose mora_type = 6
        sum_inter = in_f // r
        rb1 = in_f // r if in_f % r == 0 else in_f // r + 1

        if in_f % r != 0:
            pad_size = r - in_f % r
            x = torch.cat([x, x[..., :pad_size]], dim=-1)  # [32, 32, 960]
            sum_inter += 1
        in_x = x.view(*x.shape[:-1], sum_inter, r)  # [32, 32, 3, 256]

        if not hasattr(self, 'cos') and not hasattr(self, 'sin'):
            inv_freq = 1.0 / (10000 ** (torch.arange(0, r, 2).float() / r))
            t = torch.arange(rb1)
            freqs = torch.outer(t, inv_freq)
            emb = torch.cat((freqs, freqs), dim=-1)
            self.cos = emb.cos().unsqueeze(0).to(x.device).to(x.dtype)
            self.sin = emb.sin().unsqueeze(0).to(x.device).to(x.dtype)

        rh_in_x = torch.cat((-in_x[..., r // 2:], in_x[..., :r // 2]), dim=-1)
        # rh_in_x 最后一个维度的前 r//2 个元素是 in_x 后半部分的负值, rh_in_x 最后一个维度的后 r//2 个元素是 in_x 前半部分的原值
        in_x = in_x * self.cos + rh_in_x * self.sin  # [32, 32, 3, 256]

        # rearrange features
        b, c, h, w = in_x.shape  # [32, 32, 3, 256]
        in_x = in_x.view(b, c*h, w).permute(0, 2, 1)  # [32, 256, 96]

        '''apply lora_A'''
        out_x = self.lora_A['default'](in_x)  # [32, 256, 96]
        out_x = out_x.view(b, c, h, w)  # [32, 32, 3, 256]

        '''apply decompression after lora_A'''
        # suppose mora_type = 6
        out_x = out_x.view(*x.shape[:-1], -1)[..., :out_f]  # [32, 32, 768]
        if out_x.shape[-1] < out_f:
            repeat_time = out_f // out_x.shape[-1]
            if out_f % out_x.shape[-1] != 0:
                repeat_time += 1
            out_x = torch.cat([out_x] * repeat_time, dim=-1)[..., :out_f]  # [32, 32, 2304]

        return result + out_x

class VectorMoRAInitializer:
    def __init__(self, model, base_rank=8, mora_rank_coefficients=None, lora_dropout=0.01):
        self.model = model
        self.base_rank = base_rank
        self.lora_dropout = lora_dropout
        if mora_rank_coefficients is None:
            self.mora_rank_coefficients = [32, 32, 30, 30, 28, 28, 26, 26, 24, 24, 22, 22]
        else:
            self.mora_rank_coefficients = mora_rank_coefficients

    def calculate_mora_ranks(self):
        return [self.base_rank * coeff for coeff in self.mora_rank_coefficients]

    def initialize_mora(self):
        mora_ranks = self.calculate_mora_ranks()

        for param in self.model.transformer.parameters():
            param.requires_grad = False

        for t_layer_i, blk in enumerate(self.model.transformer.h):
            w_qkv = blk.attn.c_attn
            mora_rank = mora_ranks[t_layer_i]
            blk.attn.c_attn = Vector_MoRA(w_qkv, mora_rank, self.lora_dropout) # add vec_mora to blk.attn.c_attn

        print("Vector MoRA params initialized!")
        return self.model

class PitVQAGen(nn.Module):
    def __init__(self, base_rank=8, mora_rank_coefficients=None):
        super(PitVQAGen, self).__init__()

        # visual encoder
        model_name = "google/vit-base-patch16-224-in21k"
        self.visual_encoder = ViTModel.from_pretrained(model_name)

        # tokenizer
        self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        self.tokenizer.pad_token = self.tokenizer.eos_token  # end of string

        # text encoder
        # config = BlipConfig.from_pretrained("Salesforce/blip-vqa-base")
        # self.text_encoder = BlipTextModel(config.text_config, add_pooling_layer=False)
        self.text_encoder = BlipTextModel.from_pretrained("Salesforce/blip-vqa-base")

        # modify embedding layer
        new_vocab_size = len(self.tokenizer)
        embedding_dim = self.text_encoder.embeddings.word_embeddings.embedding_dim
        self.text_encoder.embeddings.word_embeddings = nn.Embedding(new_vocab_size, embedding_dim)  # He init

        # gpt2 decoder
        self.gpt = GPT2LMHeadModel.from_pretrained('gpt2')
        if mora_rank_coefficients is None:
            mora_rank_coefficients = [32, 32, 30, 30, 28, 28, 26, 26, 24, 24, 22, 22]
            self.gpt = VectorMoRAInitializer(self.gpt, base_rank=base_rank,
                                             mora_rank_coefficients=mora_rank_coefficients).initialize_mora()
        else:
            self.gpt = VectorMoRAInitializer(self.gpt, base_rank=base_rank,
                                             mora_rank_coefficients=mora_rank_coefficients).initialize_mora()

    def forward(self, image, question_inputs):
        # visual encoder
        image = image.to(device)
        image_embeds = self.visual_encoder(image).last_hidden_state  # torch.Size([bs, 197, 768])
        image_atts = torch.ones(image_embeds.size()[:-1], dtype=torch.long).to(image.device)  # torch.Size([bs, 197])

        question_input_ids = question_inputs['input_ids']  # torch.Size([bs, 25])
        question_att_mask = question_inputs['attention_mask']
        print('question_input_ids:', question_input_ids.shape)
        print('image_embeds:', image_embeds.shape)
        # multimodal encoder
        text_output = self.text_encoder(input_ids=question_input_ids,
                                        attention_mask=question_att_mask,
                                        encoder_hidden_states=image_embeds,
                                        encoder_attention_mask=image_atts,
                                        return_dict=True)
        text_embeds = text_output.last_hidden_state  # torch.Size([bs, 25, 768]), args.question_len=25
        print('text_embeds', text_embeds.shape)

        # text decoder
        gpt_output = self.gpt(inputs_embeds=text_embeds,
                              encoder_attention_mask=question_att_mask)  # torch.Size([bs, 25, 50257])
        return gpt_output.logits

### Main

In [ ]:
def train(args, train_dataloader, model, criterion, optimizer, epoch, tokenizer, device):
    model.train()
    total_loss = AverageMeter()

    for i, (images, questions, answers) in tqdm(enumerate(train_dataloader, 0), total=len(train_dataloader), desc="Training"):
        question_inputs = tokenizer(questions, padding="max_length", max_length=int(args.seq_length),
                                    return_tensors="pt", truncation=True)
        answer_inputs = tokenizer(answers, padding="max_length", max_length=int(args.seq_length),
                                  return_tensors="pt", truncation=True)

        # get logits and labels
        logits = model(image=images.to(device), question_inputs=question_inputs.to(device))
        # 文本生成模型的输出代表模型对每个位置的下一个词的预测
        labels = answer_inputs['input_ids'].to(device)

        # get shifted logits and labels
        shift_logits = logits[:, :-1, :].contiguous()
        shift_labels = labels[:, 1:].contiguous()

        # compute loss
        loss = criterion(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss.update(loss.item())
    print("\033[K\nEpoch: {}/{} Loss: {:.6f} AVG_Loss: {:.6f}".format(epoch, args.epochs, total_loss.val, total_loss.avg))

def validate(args, val_loader, model, criterion, epoch, tokenizer, device):
    references = []
    hypotheses = []

    model.eval()
    total_loss = AverageMeter()
    with torch.no_grad():
        for i, (images, questions, answers) in tqdm(enumerate(val_loader, 0), total=len(val_loader), desc="Validating"):
            question_inputs = tokenizer(questions, padding="max_length", max_length=int(args.seq_length),
                                        return_tensors="pt", truncation=True)
            answer_inputs = tokenizer(answers, padding="max_length", max_length=int(args.seq_length),
                                      return_tensors="pt", truncation=True)

            # get logits and labels
            logits = model(image=images.to(device), question_inputs=question_inputs.to(device))
            labels = answer_inputs['input_ids'].to(device)

            # get shifted logits and labels
            shift_logits = logits[:, :-1, :].contiguous()
            shift_labels = labels[:, 1:].contiguous()

            # compute loss
            loss = criterion(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
            total_loss.update(loss.item())

            # generate predicted answer
            _, predicted = torch.max(logits, dim=-1)

            # decode references and predictions
            reference_answers = tokenizer.batch_decode(labels, skip_special_tokens=True)
            predicted_answers = tokenizer.batch_decode(predicted, skip_special_tokens=True)

            # add references and hypotheses to lists
            for ref, hyp in zip(reference_answers, predicted_answers):
                references.append([ref.split()])
                hypotheses.append(hyp.split())

        # Calculate BLEU_1~4
        metrics = {}
        metrics["Bleu_1"] = corpus_bleu(references, hypotheses, weights=(1.00, 0.00, 0.00, 0.00))
        metrics["Bleu_2"] = corpus_bleu(references, hypotheses, weights=(0.50, 0.50, 0.00, 0.00))
        metrics["Bleu_3"] = corpus_bleu(references, hypotheses, weights=(0.33, 0.33, 0.33, 0.00))
        metrics["Bleu_4"] = corpus_bleu(references, hypotheses, weights=(0.25, 0.25, 0.25, 0.25))
        print(f"\033[K\nEpoch: {epoch}/{args.epochs} EVA LOSS: {total_loss.avg:.6f} "
              f"BLEU-1: {metrics['Bleu_1']:.6f} BLEU-2: {metrics['Bleu_2']:.6f} "
              f"BLEU-3: {metrics['Bleu_3']:.6f} BLEU-4: {metrics['Bleu_4']:.6f}")

    return metrics

def seed_everything(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

class Hyperparameters:
    epochs = 50
    batch_size = 32
    workers = 2
    random_seed = 42

    seq_length = 32
    base_rank = 8
    coeff = 'vec1'
    lr = 0.00002

    def __init__(self):
        pass



if __name__ == '__main__':
    # hyperparameter
    args = Hyperparameters()

    seed_everything(args.random_seed)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    start_epoch = 1
    best_epoch = [0]
    best_results = [0.0]
    epochs_since_improvement = 0

    # data location
    train_seq = [2, 3, 4, 6, 7, 9, 10, 11, 12, 14, 15]
    val_seq = [1, 5, 16]

    folder_head = '/content/EndoVis-18-VQA/seq_'
    folder_tail = '/vqa/Sentence/*.txt'

    # dataloader
    train_dataset = EndoVis18VQAGen(train_seq, folder_head, folder_tail)
    train_dataloader = DataLoader(dataset=train_dataset, batch_size=args.batch_size,
                    shuffle=True, num_workers=args.workers)
    val_dataset = EndoVis18VQAGen(val_seq, folder_head, folder_tail)
    val_dataloader = DataLoader(dataset=val_dataset, batch_size=args.batch_size,
                    shuffle=False, num_workers=args.workers)

    # init tokenizer and model
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    tokenizer.pad_token = tokenizer.eos_token

    vector = []
    if args.coeff == 'vec1':
        vector = [32, 32, 30, 30, 28, 28, 26, 26, 24, 24, 22, 22]
    elif args.coeff == 'vec2':
        vector = [64, 64, 60, 60, 56, 56, 52, 52, 48, 48, 44, 44]
    elif args.coeff == 'vec3':
        vector = [24, 24, 22, 22, 20, 20, 18, 18, 16, 16, 14, 14]
    model = PitVQAGen(base_rank=args.base_rank, mora_rank_coefficients=vector)
    model = model.to(device)

    pytorch_total_params = sum(p.numel() for p in model.parameters())
    print('model params: ', pytorch_total_params)

    # init optimizer and criterion
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
    criterion = nn.CrossEntropyLoss().to(device)

    # train and validation
    print('Start training.')
    for epoch in range(start_epoch, args.epochs+1):
        if epochs_since_improvement > 0 and epochs_since_improvement % 5 == 0:
            adjust_learning_rate(optimizer, 0.8)

        # train
        train(args, train_dataloader=train_dataloader, model=model, criterion=criterion, optimizer=optimizer,
              epoch=epoch, tokenizer=tokenizer, device=device)
        # validation
        metrics = validate(args, val_loader=val_dataloader, model=model, criterion=criterion, epoch=epoch,
                           tokenizer=tokenizer, device=device)

        if metrics["Bleu_4"] >= best_results[0]:
            epochs_since_improvement = 0
            best_results[0] = metrics["Bleu_4"]
            best_epoch[0] = epoch
            print(f'Best epoch: {epoch}, Best Bleu_4: {metrics["Bleu_4"]}')
            # save_clf_checkpoint(args.checkpoint_dir, epoch, epochs_since_improvement,
            #                     model, optimizer, best_results[0], final_args=None)
        else:
            epochs_since_improvement += 1
            print("\nEpochs since last improvement: %d\n" % (epochs_since_improvement,))
    print('End training.')

Total files: 1560 | Total question: 10574
Total files: 447 | Total question: 3216


Some weights of BlipTextModel were not initialized from the model checkpoint at Salesforce/blip-vqa-base and are newly initialized: ['embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.crossattention.output.LayerNorm.bias', 'encoder.layer.0.crossattention.output.LayerNorm.weight', 'encoder.layer.0.crossattention.output.dense.bias', 'encoder.layer.0.crossattention.output.dense.weight', 'encoder.layer.0.crossattention.self.

Vector MoRA params initialized!
model params:  364401920
Start training.


Training:   0%|          | 0/331 [00:00<?, ?it/s]

question_input_ids: torch.Size([32, 32])
image_embeds: torch.Size([32, 197, 768])
text_embeds torch.Size([32, 32, 768])


...

question_input_ids: torch.Size([32, 32])
image_embeds: torch.Size([32, 197, 768])
text_embeds torch.Size([32, 32, 768])


Training:   1%|          | 3/331 [00:10<15:19,  2.80s/it]

question_input_ids: torch.Size([32, 32])
image_embeds: torch.Size([32, 197, 768])
text_embeds torch.Size([32, 32, 768])


Training:   1%|          | 3/331 [00:11<21:32,  3.94s/it]


KeyboardInterrupt: 